## install requierd packages: 

In [1]:
# !pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

In [2]:
!unzip /content/drive/MyDrive/snappfood_reviews.zip # unzip data from google drive

Archive:  /content/drive/MyDrive/snappfood_reviews.zip
replace snappfood/dev.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace snappfood/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace snappfood/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
#import libraries

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [4]:
train = pd.read_csv('/content/snappfood/train.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
test = pd.read_csv('/content/snappfood/test.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
dev = pd.read_csv('/content/snappfood/dev.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
print('{} and {} are numbers of training and development(validation) data and {} is number of test data.'.format(len(train), len(dev), len(test)), "\n")
train.head()

56700 and 6300 are numbers of training and development(validation) data and 7000 is number of test data. 



,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [5]:
train = train[['comment', 'label_id']]
test = test[['comment', 'label_id']]
dev = dev[['comment', 'label_id']]
train.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [6]:
dev.isnull().sum()  # checking for NaN

comment     0
label_id    0
dtype: int64

In [7]:
(pd.concat([train, dev, test])).groupby(['label_id']).count() # check if dataset is balanced or not

,comment
label_id,
0,35000
1,35000


## Preprocessing text

In [8]:
alphabet_dict={'ﺎ':'ا','ﺍ':'ا','ﺑ':'ب','ﺒ':'ﺐ','ﺐ':'ب','ﺏ':'ب','ﺗ':'ت','ﺘ':'ت','ﺖ':'ت','ﺕ':'ت','ﺛ':'ث','ﺜ':'ث','ﺚ':'ث','ﺙ':'ث','ﺟ':'ج','ﺠ':'ج',
               'ﺞ':'ج','ﺝ':'ج','ﺣ':'ح','ﺤ':'ح','ﺢ':'ح','ﺡ':'ح','ﺧ':'خ','ﺨ':'خ','ﺦ':'خ','ﺥ':'خ','ﺪ':'د','ﺩ':'د','ﺬ':'ذ','ﺫ':'ذ','ﺮ':'ر','ﺭ':'ر','ﺰ':'ز',
               'ﺯ':'ز','ﺳ':'س','ﺴ':'س','ﺲ':'س','ﺱ':'س','ﺷ':'ش','ﺸ':'ش','ﺶ':'ش','ﺵ':'ش','ﺻ':'ص','ﺼ':'ص','ﺺ':'ص','ﺹ':'ص','ﺿ':'ض',
               'ﻀ':'ض','ﺾ':'ض','ﺽ':'ض','ﻃ':'ط','ﻄ':'ط','ﻂ':'ط','ﻁ':'ط','ﻇ':'ظ','ﻈ':'ظ','ﻆ':'ظ','ﻅ':'ظ','ﻋ':'ع','ﻌ':'ع','ﻊ':'ع','ﻉ':'ع',
               'ﻏ':'غ','ﻐ':'غ','ﻎ':'غ','ﻍ':'غ','ﻓ':'ف','ﻔ':'ف','ﻒ':'ف','ﻑ':'ف','ﻗ':'ق','ﻘ':'ق','ﻖ':'ق','ﻕ':'ق','ﻛ':'ک','ﻜ':'ک','ﻚ':'ک','ﻙ':'ک',
               'ﻟ':'ل','ﻠ':'ل','ﻞ':'ل','ﻝ':'ل','ﻣ':'م','ﻤ':'م','ﻢ':'م','ﻡ':'م','ﻧ':'ن','ﻨ':'ن','ﻦ':'ن','ﻥ':'ن','ﻫ':'ه','ﻬ':'ه','ﻪ':'ه','ﻩ':'ه','ﻮ':'و',
               'ﻭ':'و','ﻳ':'ی','ﻴ':'ی','ﻲ':'ی','ﻱ':'ی','ﺂ':'آ','ﺁ':'آ','ﺔ':'ه','ﺓ':'ه','ﻰ':'ی','ﻯ':'ی','ئ':'ی','ﭖ':'پ','ﭻ':'چ','ڗ':'ژ','ٶ':'و','ۯ':'ژ',
               'ٱ':'ا','ﺅ':'و','ﮔ':'گ','ﯿ':'ی','ى':'ی','ۃ':'ه','ە':'ه','ة':'ه','ہ':'ه','أ':'ا','ك':'ک','إ':'ا','ۀ':'ه','ھ':'ه','ۆ':'و','ﮐ':'ک','ﭘ':'پ','ﺐ':'ب',
               'ﮏ':'ک','ﭼ':'چ','ﯾ':'ی','ﮕ':'گ','ﮋ':'ژ','ﮑ':'ک','ﭽ':'چ','ۍ':'ی','ﯼ':'ی','ﺆ':'و','ﺌ':'ی','ﺄ':'ا','ﭗ':'پ','ﮎ':'ک','ﮒ':'گ','ګ':'گ',
               'ﭙ':'پ','ﺋ':'ی','ﮓ':'گ','ي':'ی','ﯽ':'ی'}

# normal alphabets :
def edit_alphabet(text):
  output=''
  for t in text:
    if t in alphabet_dict.keys():
      output = output + alphabet_dict.get(t)
    else:
      output = output + t
  return output

# cleaning
def clean_text(t):

  t = edit_alphabet(t)
  t = re.sub(r'[^آابپتثئجچحخدذرزژسشصضطظعغفقکگلمنوهی\d\s]+','',t)
  t = re.sub(r'\d+' , ' N ' , t)
  t = re.sub(r'  ', ' ', t)
  t = re.sub(r'_' , '' , t)
  t = re.sub(r'\.+' , '.' , t)
  t = re.sub(r'\n' , ' ' , t)
  t = re.sub(r'\r|\u2003|\u200a|\u2009|\u3000|\x1f|\u2028|\u2029|\x0c|\u2005|\x85' ,' ' ,t)

  return t



def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def apply_cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing weird patterns
    weird_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = weird_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    # edit alphabets and clean any other unusefull character
    text = clean_text(text)

    
    return text

In [9]:
# apply preprocessing on dataset : 
train['cleaned_comment'] = train['comment'].apply(apply_cleaning)
test['cleaned_comment'] = test['comment'].apply(apply_cleaning)
dev['cleaned_comment'] = dev['comment'].apply(apply_cleaning)

train = train[['cleaned_comment', 'label_id']]
test = test[['cleaned_comment', 'label_id']]
dev = dev[['cleaned_comment', 'label_id']]
train.head()

,cleaned_comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود N ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره فقط ...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود,0


In [10]:
x_train, y_train = train['cleaned_comment'].values.tolist(), train['label_id'].values.tolist()
x_test, y_test = test['cleaned_comment'].values.tolist(), test['label_id'].values.tolist()
x_dev, y_dev = dev['cleaned_comment'].values.tolist(), dev['label_id'].values.tolist()

In [11]:
# Vectorize text reviews to numbers (using tf-idf vectorizer)
vec = TfidfVectorizer()
X_train = vec.fit_transform(x_train)
X_test = vec.transform(x_test)
print("size of vocabulary is: \n", len(vec.vocabulary_))

size of vocabulary is: 
 26357


In [12]:
# Naive bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [13]:
model.score(X_test, y_test)

0.8318571428571429

In [14]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.88      0.77      0.82      3500
           1       0.79      0.90      0.84      3500

    accuracy                           0.83      7000
   macro avg       0.84      0.83      0.83      7000
weighted avg       0.84      0.83      0.83      7000



check

In [16]:
model.predict(vec.transform(['غذاش سرد بود.']))

array([1])

In [17]:
model.predict(vec.transform(['بستنی آب شده بود']))

array([1])